In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, json
import pandas as pd
import subprocess
import time
from random import sample
from os import listdir
from os.path import isfile, join
import re
import praw
from supabase import create_client, Client

In [3]:
creds = json.load(open(".db-creds.json"))

url = creds["SUPABASE_URL"]
key = creds["SUPABASE_KEY"]
supabase = create_client(url, key)

In [4]:
reddit_creds = json.load(open(".reddit_creds.json", "r"))
reddit = praw.Reddit(
    client_id=reddit_creds["client_id"],
    client_secret=reddit_creds["client_secret"],
    password=reddit_creds["password"],
    user_agent=reddit_creds["user_agent"],
    username=reddit_creds["username"],
)

In [5]:
r_all = reddit.subreddit('all')

In [9]:
def get_comment_data(comment, usernames, posts_result):    
    try:
        username = str(comment.author.name)
        if username[0:2] == '0x':
            username = username[2:]
        profile_pic = comment.author.icon_img
    except:
        username = "removed"
        profile_pic = "removed"

    if username not in usernames:
        user_data = supabase.table("users").insert({"username": username, "profile_pic": profile_pic}).execute().data
        user_id = user_data[0]['id']
    else:
        user_id = usernames[username]

    comment_data = {
        "post_id": posts_result["id"],
        "user_id": user_id,
        "body": comment.body,
        "depth": comment.depth,
        "reddit_id": comment.name,
        "reddit_upvotes": comment.ups,
        "reddit_downvotes": comment.downs,
    }
    return comment_data

In [10]:
def get_comments(comment_list, comment_reddit_ids, usernames, posts_result):
    comments = []
    for comment in comment_list:
        if comment.name in comment_reddit_ids:
            continue

        if type(comment) != praw.models.reddit.comment.Comment:
            print(type(comment))
            comments.extend(get_comments(comment.comments(), comment_reddit_ids, usernames, posts_result))
        else:
            comment_data = get_comment_data(comment, usernames, posts_result)
            comments.append(comment_data)
    return comments

# DB writer

In [ ]:
minted_time = 0
last_block = 0
while True:
    # Mint reddit posts every hour
    if (time.time() - minted_time) / 60 > 30:
        post_reddit_ids = [row['reddit_id'] for row in supabase.table("posts").select("reddit_id").execute().data]
        comment_reddit_ids = [row['reddit_id'] for row in supabase.table("comments").select("reddit_id").execute().data]
        categories = {row['name']: row['id'] for row in supabase.table("categories").select("id, name").execute().data}
        usernames = {row['username']: row['id'] for row in supabase.table("users").select("id, username").execute().data}
        
        posts = []
        for i, post in enumerate(r_all.top(time_filter='hour')):
            if post.name in post_reddit_ids:
                continue
                
            if i > 10:
                continue
                     
            category = post.subreddit.display_name
            if category not in categories:
                category_data = supabase.table("categories").insert({"name": category}).execute().data
                category_id = category_data[0]['id']
            else:
                category_id = categories[category]

            try:
                username = str(post.author.name)
                if username[0:2] == '0x':
                    username = username[2:]
                profile_pic = post.author.icon_img
            except:
                username = "removed"
                profile_pic = "removed"

            if username not in usernames:
                user_data = supabase.table("users").insert({"username": username, "profile_pic": profile_pic}).execute().data
                user_id = user_data[0]['id']
            else:
                user_id = usernames[username]

            post_data = {
                "category_id": category_id,
                "user_id": user_id,
                "title": post.title,
                "body": post.selftext,
                "url": post.url,
                "is_nsfw": post.over_18,
                "reddit_id": post.name,
                "reddit_upvotes": post.ups,
                "reddit_downvotes": post.downs,
            }
            posts.append(post_data)
            posts_result = supabase.table("posts").insert(post_data).execute().data[0]

            comments = get_comments(post.comments.list(), comment_reddit_ids, usernames, posts_result)
                
            supabase.table("comments").insert(comments).execute().data
            print(f"added {len(comments)} comments")
        
        print(f"added {len(posts)} posts")
        print('')
        minted_time = time.time()
    
    time.sleep(1)

added 245 comments
added 17 comments
added 57 comments
added 248 comments
added 1 comments
added 122 comments
added 169 comments
added 49 comments
added 14 comments
added 8 comments
added 7 comments
added 57 comments
added 4 comments
added 57 comments
added 4 comments
added 58 comments
added 5 comments
added 9 comments
added 15 comments
added 5 comments
added 98 comments
added 54 comments
added 22 comments
added 58 comments
added 9 comments


2022-08-11 10:27:27,943:WARNING - Retrying due to ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='oauth.reddit.com', port=443): Read timed out. (read timeout=16.0)")) status: GET https://oauth.reddit.com/user/TotallyNotADoc/about/


added 27 comments


2022-08-11 10:28:17,666:WARNING - Retrying due to ReadTimeout(ReadTimeoutError("HTTPSConnectionPool(host='oauth.reddit.com', port=443): Read timed out. (read timeout=16.0)")) status: GET https://oauth.reddit.com/user/Surfsupforthesummer/about/


added 107 comments
added 14 comments
added 1 comments
added 8 comments
added 45 comments
added 3 comments
added 3 comments
added 21 comments
added 10 comments
added 10 comments
added 5 comments
added 13 comments
added 17 comments
added 60 comments
added 40 comments
added 35 comments
added 5 comments
added 4 comments
added 3 comments
added 25 comments
added 122 comments
added 17 comments
added 2 comments
added 20 comments
added 4 comments
added 76 comments
added 10 comments
added 7 comments
added 8 comments
added 107 comments
added 17 comments
added 12 comments
added 76 comments
added 10 comments
added 50 comments
added 8 comments
added 38 comments
added 1 comments
added 3 comments
added 47 comments
added 4 comments
added 3 comments
added 27 comments
added 89 comments
added 10 comments
added 33 comments
added 35 comments
added 4 comments
added 0 comments
added 29 comments
added 45 comments
added 6 comments
added 3 comments
added 4 comments
added 48 comments
added 13 comments
added 4 com

added 44 comments
added 4 comments
added 8 comments
added 26 comments
added 11 comments
added 0 comments
added 155 comments
added 19 comments
added 42 comments
added 2 comments
added 7 comments
added 12 comments
<class 'praw.models.reddit.more.MoreComments'>
added 30 comments
added 2 comments
added 19 comments
added 16 comments
added 36 comments
added 8 comments
added 5 comments
added 105 comments
added 9 comments
added 12 comments
added 4 comments
added 86 comments
added 18 comments
added 0 comments
added 14 comments
added 15 comments
added 8 comments
added 63 comments
added 4 comments
added 12 comments
added 0 comments
added 133 comments
added 1 comments
added 0 comments
added 55 comments
added 37 comments
<class 'praw.models.reddit.more.MoreComments'>
<class 'praw.models.reddit.more.MoreComments'>
<class 'praw.models.reddit.more.MoreComments'>
<class 'praw.models.reddit.more.MoreComments'>
<class 'praw.models.reddit.more.MoreComments'>
<class 'praw.models.reddit.more.MoreComments'>


In [18]:
post_reddit_ids = [row['reddit_id'] for row in supabase.table("posts").select("reddit_id").execute().data]

In [19]:
post.name in post_reddit_ids

False

In [20]:
"t3_wgryek" in post_reddit_ids

False

In [16]:
post_reddit_ids

['t3_wgg1ov',
 't3_wgg6j4',
 't3_wgg0hh',
 't3_wgfvi9',
 't3_wgg0bb',
 't3_wgfzxs',
 't3_wgfyxf',
 't3_wgfyd0',
 't3_wgg8n7',
 't3_wgg2n8',
 't3_wgggjd',
 't3_wgg1l4',
 't3_wgg0nl',
 't3_wgg2e2',
 't3_wgg5fj',
 't3_wgfx9w',
 't3_wgg442',
 't3_wgg8bb',
 't3_wggk02',
 't3_wgga3m',
 't3_wggdvg',
 't3_wggsin',
 't3_wggkkt',
 't3_wggx5l',
 't3_wgh3iq',
 't3_wggiyj',
 't3_wggiwx',
 't3_wggkt3',
 't3_wgh4a8',
 't3_wggoj7',
 't3_wgh2bn',
 't3_wggwzo',
 't3_wggwnj',
 't3_wggl11',
 't3_wgh2wo',
 't3_wggksk',
 't3_wggyin',
 't3_wggso0',
 't3_wggtaa',
 't3_wggl7h',
 't3_wggj0g',
 't3_wghfhe',
 't3_wggtqx',
 't3_wggy34',
 't3_wggjn7',
 't3_wggo4z',
 't3_wggjsj',
 't3_wggrqo',
 't3_wgh42d',
 't3_wggj0t',
 't3_wggik3',
 't3_wggi2y',
 't3_wggwpr',
 't3_wggt79',
 't3_wgglh5',
 't3_wggpsw',
 't3_wggnhx',
 't3_wggqq7',
 't3_wggx0i',
 't3_wgh0zt',
 't3_wggm26',
 't3_wgh9rn',
 't3_wggpzl',
 't3_wgh47k',
 't3_wghwp3',
 't3_wghwif',
 't3_wghsqb',
 't3_wghyek',
 't3_wghttz',
 't3_wghkhy',
 't3_wghpw6',
 't3_w

In [17]:
post_data

{'category_id': 36,
 'user_id': 14257,
 'title': '+36 from Good old Germany',
 'body': '',
 'url': 'https://i.redd.it/9ld497yzgvf91.jpg',
 'is_nsfw': False,
 'reddit_id': 't3_wgryek',
 'reddit_upvotes': 267,
 'reddit_downvotes': 0}